## Blue Team Accuracy and Confidence with e = 0.1 and 0.25

Importing the necessary modules like theano, tensor, sandbox, numpy, pickle, matplotlib

In [1]:
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np
import os
import pickle as cPickle 
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
#from pprint import pprint

Defining the required functions to make use of weights, RMS prop, Neural Network model, one hot and MNIST train, test and digits

In [2]:
################## Taken from https://github.com/robertlacok/mnist-adversarial-examples #########################
################### Taken from https://github.com/Newmu/Theano-Tutorials/ #######################################
# Convert into correct type for theano
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

# Weights are shared theano variables
def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

# RMSProp to update weights
def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

# Dropout regularization 
def dropout(X, p=0.):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

# Neural network model, 3 fully connected layers
def model(X, w_h, w_h2, w_o, p_drop_input, p_drop_hidden):
	# Input layer: dropout + relu 
    X = dropout(X, p_drop_input)
    h = T.nnet.relu(T.dot(X, w_h))
	
	# Hidden layer: dropout + relu 
    h = dropout(h, p_drop_hidden)
    h2 = T.nnet.relu(T.dot(h, w_h2))
	
	# Output layer: dropout + softmax 
    h2 = dropout(h2, p_drop_hidden)
    py_x = T.nnet.softmax(T.dot(h2, w_o))
    return h, h2, py_x

def plot_mnist_digit(image1, image2, name1, name2):
    global count_attack
    image1 = np.reshape(image1,[1,784])
    image2 = np.reshape(image2,[1,784])
    #print 'test image confidence' , np.mean(predict_conf(image1)), 'adversarial image confidence', 
    #np.mean(predict_conf(image2))
    #if (predict(image1) != predict(image2)):
	#count_attack = count_attack + 1
    
def one_hot(x,n):
	if type(x) == list:
		x = np.array(x)
	x = x.flatten()
	o_h = np.zeros((len(x),n))
	o_h[np.arange(len(x)),x] = 1
	return o_h

def mnist(ntrain=60000,ntest=10000,onehot=True):
	data_dir = os.path.join(datasets_dir,'mnist/')
	fd = open(os.path.join(data_dir,'train-images-idx3-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	trX = loaded[16:].reshape((60000,28*28)).astype(float)

	fd = open(os.path.join(data_dir,'train-labels-idx1-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	trY = loaded[8:].reshape((60000))

	fd = open(os.path.join(data_dir,'t10k-images-idx3-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	teX = loaded[16:].reshape((10000,28*28)).astype(float)

	fd = open(os.path.join(data_dir,'t10k-labels-idx1-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	teY = loaded[8:].reshape((10000))

	trX = trX/255.
	teX = teX/255.

	trX = trX[:ntrain]
	trY = trY[:ntrain]

	teX = teX[:ntest]
	teY = teY[:ntest]

	if onehot:
		trY = one_hot(trY, 10)
		teY = one_hot(teY, 10)
	else:
		trY = np.asarray(trY)
		teY = np.asarray(teY)

	return trX,teX,trY,teY

* Here, we are loading the MNIST data into the media/datasets folder
* Initialized weights using init_weights
* Cost function is defined and Theano expressions are updated
* L1 and L2 equations are presented

In [3]:
######## Training sizes and parameters taken from https://github.com/robertlacok/mnist-adversarial-examples #######

datasets_dir = 'media/datasets/'
srng = RandomStreams()
TRAINING = True

print('MNIST data Loaded')
Xtr, Xts, ytr, yts = mnist(onehot=True)

# Initialize theano variables for X, Y, and shared variables for weights
X = T.fmatrix()
Y = T.fmatrix()

if TRAINING:
    # For training of the net, we initialize weights to random values
    w_h = init_weights((784, 625))
    w_h2 = init_weights((625, 625))
    w_o = init_weights((625, 10))
    params = [w_h, w_h2, w_o]
else:
    # To run experiments, just read weights we learned before
    w_h, w_h2, w_o = params

# Dropout model for training
noise_h, noise_h2, noise_py_x = model(X, w_h, w_h2, w_o, 0.2, 0.5)
# Use all-weights model for prediction
h, h2, py_x = model(X, w_h, w_h2, w_o, 0., 0.)
y_x = T.argmax(py_x, axis=1)

# To find confidence of test set use the following value of y_x
y_x1 = T.max(py_x, axis = 1)
# Define cost and update theano expressions

l1 = abs(w_h).sum() + abs(w_h2).sum() + abs(w_o).sum()
l2 = (w_h**2).sum() + (w_h2**2).sum() + (w_o**2).sum()

MNIST data Loaded


* We are varying L1 and L2 Coefficients for a set of values: 0.1, 0.0001, 0.00001
* Defined Cost and Train and predicted using theano functions
* Trained MNISt data for 5 Epochs
* Accuracy, Confidence for different L1, L2 Coefficients are presented

* EPS values are taken as 0.1 and 0.25
* Adversarial Set Accuracy and Confidence are presented

In [4]:
################################################# Our Part of the code #######################################################

#==========================Training for blue team==========================#

#=================== Parameters to change ===============================#
l1coef = [ 0.0, 0.0001, 0.00001 ] 
l2coef = [ 0.0, 0.0001, 0.00001 ] 
#=======================================================================#

for l in l1coef:
    for j in l2coef:
        print("l1coef = %f, l2coef = %f" %(l,j))
        cost = T.mean(T.nnet.categorical_crossentropy(noise_py_x, Y)) + l * l1 + j * l2
        updates = RMSprop(cost, params, lr=0.001)
        train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)
        predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)
        predict_conf = theano.function(inputs=[X], outputs=y_x1, allow_input_downcast=True)
        
        if TRAINING:
            # Train in 5 epochs
            for k in range(5):
                # Select minibatch and train
                for start, end in zip(range(0, len(Xtr), 128), range(128, len(Xtr), 128)):
                    cost = train(Xtr[start:end], ytr[start:end])
            print("Accuracy is:    ",np.mean(np.argmax(ytr, axis=1) == predict(Xtr)))
            print("Confidence is:    ", np.mean(predict_conf(Xtr)))

        ###### Creating Adversarial Examples ######
        cost_ad = T.mean(T.nnet.categorical_crossentropy(py_x, Y))
        get_grad = theano.function(inputs=[X, Y], outputs=T.grad(cost_ad, X), allow_input_downcast=True)
        
        #============================Parameter to change ========================================#
        eps_values = [0.10, 0.25]  
        #========================================================================================#

        for EPS in eps_values:
            eps = EPS
            adX = []
            for i in range(len(Xts)):
                gs = get_grad(Xts[i:i+1], yts[i:i+1]).T[:,0]
                img_ad = Xts[i] + eps * np.sign(gs) 
                adX.append(img_ad)

            # Find accuracy of the classifier on the test set and adversarial set
            pred_yts = predict(Xts)
            print('Adversarial set Accuracy, e=', eps, ':			', np.mean(np.argmax(yts, axis=1) == predict(adX)))
            print('Adversarial set Confidence:				', np.mean(predict_conf(adX)))
            print('================================================================================')

l1coef = 0.000000, l2coef = 0.000000
Accuracy is:     0.9755
Confidence is:     0.981086115583
Adversarial set Accuracy, e= 0.1 :			 0.3709
Adversarial set Confidence:				 0.766548066502
Adversarial set Accuracy, e= 0.25 :			 0.0112
Adversarial set Confidence:				 0.748038334176
l1coef = 0.000000, l2coef = 0.000100
Accuracy is:     0.9798
Confidence is:     0.981353846668
Adversarial set Accuracy, e= 0.1 :			 0.4488
Adversarial set Confidence:				 0.738770862935
Adversarial set Accuracy, e= 0.25 :			 0.0135
Adversarial set Confidence:				 0.647579388018
l1coef = 0.000000, l2coef = 0.000010
Accuracy is:     0.9835
Confidence is:     0.988574713089
Adversarial set Accuracy, e= 0.1 :			 0.4502
Adversarial set Confidence:				 0.776442656081
Adversarial set Accuracy, e= 0.25 :			 0.014
Adversarial set Confidence:				 0.678793001428
l1coef = 0.000100, l2coef = 0.000000
Accuracy is:     0.9682
Confidence is:     0.960986890531
Adversarial set Accuracy, e= 0.1 :			 0.6108
Adversarial set Confi